# Давайте обучим собачку Huggy бегать за палочкой

### Окружение 🎮

- Спасибо за окружение Huggy the Dog  [Thomas Simonini](https://twitter.com/ThomasSimonini) based on [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)

### Библиотека, которой воспользуемся 📚

- [MLAgents](https://github.com/Unity-Technologies/ml-agents)

## Цели этой практики 🏆

- Понимание абстракций **функции награды агента, пространства состояния и действий агента**.
- Обучим собачку приносить палочку


## Что необходимо для выполнения этой практики 📚

- Python **3.10.12**
- Свободная память (около 4 Гб)
- GPU или CPU для обучения нашей собачки
- Аккаунт на HuggingFace


In [ ]:
# Предлагаю проверить версию нашу версию Питона
!python --version
# Должно быть 3.10.12

# Давайте начнем!

In [ ]:
%%capture
# Загрузим библиотеку для работы с агентом
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

### Если Вы выполняете этот ноутбук в Колабе, то можете пройти по инструкции ниже и без проблем скачать и настроить необходимую версию питона

In [ ]:
!pip install virtualenv
!virtualenv myenv

!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

!source /usr/local/bin/activate
!conda install -q -y --prefix /usr/local python=3.10.12 ujson  # Specify the version here

!export PYTHONPATH=/usr/local/lib/python3.10/site-packages/
!export CONDA_PREFIX=/usr/local/envs/myenv

In [ ]:
# Проверем еще раз
!python --version

## Вернемся к настройке

Установим нашу библиотеку с МЛ агентами 🔽

In [ ]:
%%capture
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

## Установим наше окружения, которое будет средой обучения для нашего агента

In [ ]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

In [ ]:
!wget "https://github.com/huggingface/Huggy/raw/main/Huggy.zip" -O ./trained-envs-executables/linux/Huggy.zip

In [ ]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

Проверим, что у нас есть доступ к окружению

In [ ]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy

## Краткий обзор
### Пространство состояний: что "воспринимает" Huggy

Huggy не "видит" окружающую среду. Вместо этого мы предоставляем ему информацию о среде:

- Положение цели (палки).
- Относительное положение между ним и целью.
- Ориентация его ног.

Имея всю эту информацию, Huggy **может решить, какое действие предпринять следующим, чтобы достичь своей цели**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.jpg" alt="Huggy" width="100%">

---

### Пространство действий: какие движения может выполнять Huggy

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-action.jpg" alt="Huggy action" width="100%">

**Суставные моторы управляют ногами Huggy**. Это означает, что для достижения цели Huggy должен **научиться правильно вращать моторы каждой из своих ног, чтобы двигаться**.

---

### Функция вознаграждения

Функция вознаграждения разработана так, чтобы **Huggy мог достичь своей цели**: добраться до палки.

Напомним, что одной из основ обучения с подкреплением (*Reinforcement Learning*) является *гипотеза вознаграждения*: цель может быть описана как **максимизация ожидаемого кумулятивного вознаграждения**.

В данном случае наша цель заключается в том, чтобы Huggy **двигался к палке, но при этом не вращался слишком сильно**. Следовательно, наша функция вознаграждения должна отражать эту цель.

Наша функция вознаграждения:

- **Бонус за ориентацию**: мы вознаграждаем Huggy за приближение к цели.
- **Штраф за время**: фиксированный штраф, накладываемый на каждое действие, чтобы заставить его достичь палки как можно быстрее.
- **Штраф за вращение**: мы наказываем Huggy, если он слишком сильно вращается или поворачивается слишком быстро.
- **Вознаграждение за достижение цели**: мы вознаграждаем Huggy за достижение цели.

---

Этот текст описывает ключевые аспекты обучения с подкреплением (*Reinforcement Learning*), такие как пространство состояний, пространство действий и функция вознаграждения, которые используются для обучения агента (в данном случае Huggy) достижению поставленной задачи.

## Создание конфигурационного файла для Huggy

- В ML-Agents **гиперпараметры обучения определяются в файлах `config.yaml`.**

- В рамках этого руководства мы не будем изменять гиперпараметры, но если вы хотите поэкспериментировать, рекомендуется попробовать изменить некоторые из них. Unity предоставляет [отличную документацию, объясняющую каждый из параметров](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md).

- Нам необходимо создать конфигурационный файл для Huggy.

  - Для этого нажмите на иконку папки в левой части экрана.

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create_file.png" alt="Create file" width="10%">

  - Перейдите в папку `/content/ml-agents/config/ppo`.
  - Щелкните правой кнопкой мыши и создайте новый файл с именем `Huggy.yaml`.

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create-huggy.png" alt="Create huggy.yaml" width="20%">

- Скопируйте и вставьте содержимое ниже:

In [ ]:
behaviors:
  Huggy:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: true
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.995
        strength: 1.0
    checkpoint_interval: 200000
    keep_checkpoints: 15
    max_steps: 2e6
    time_horizon: 1000
    summary_freq: 50000

## Настройка гиперпараметров

- **Если вы хотите изменить гиперпараметры**, в Google Colab вы можете открыть файл конфигурации по следующему пути: `/content/ml-agents/config/ppo/Huggy.yaml`.

- Например, **если вы хотите сохранять больше моделей в процессе обучения** (сейчас мы сохраняем модель каждые 200 000 шагов обучения), вам нужно изменить следующие параметры:
  - `checkpoint_interval`: Количество шагов обучения между сохранением каждой контрольной точки (checkpoint).
  - `keep_checkpoints`: Максимальное количество сохраняемых контрольных точек.

=> Учтите, что **уменьшение значения `checkpoint_interval` приведет к сохранению большего количества моделей и, как следствие, увеличит время загрузки на Hugging Face Hub**.

Теперь мы готовы к обучению нашего агента 🔥.

## Обучение нашего агента

Чтобы обучить нашего агента, нам нужно **запустить `mlagents-learn` и указать исполняемый файл, содержащий окружение.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mllearn.png" alt="ml learn function" width="100%">

С ML Agents мы запускаем скрипт обучения. Мы определяем четыре параметра:

1. `mlagents-learn <config>`: путь к файлу конфигурации с гиперпараметрами.
2. `--env`: путь к исполняемому файлу окружения.
3. `--run-id`: имя, которое вы хотите присвоить вашему запуску обучения.
4. `--no-graphics`: отключает визуализацию во время обучения.

Обучите модель и используйте флаг `--resume`, чтобы продолжить обучение в случае прерывания.

> При первом использовании `--resume` может возникнуть ошибка. Попробуйте запустить блок снова, чтобы обойти её.

**Дальнейшее обучение займет около 40 минут на GPU**

In [ ]:
!mlagents-learn ./config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy2" --no-graphics

## Загрузка агента в 🤗 Hub

- Теперь, когда мы обучили нашего агента, мы **готовы загрузить его в Hugging Face Hub, чтобы можно было взаимодействовать с Huggy прямо в браузере🔥.**

## Подготовка к загрузке модели в HF

Чтобы поделиться своей моделью с сообществом, необходимо выполнить три шага:

1️⃣ (Если вы еще этого не сделали) Создайте аккаунт на Hugging Face ➡ [https://huggingface.co/join](https://huggingface.co/join)

2️⃣ Авторизуйтесь и затем вам нужно сохранить ваш токен аутентификации с сайта Hugging Face.
- Создайте новый токен ([https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)) **с ролью write (запись)**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- Скопируйте токен.
- Запустите ячейку ниже и вставьте токен.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="./results/Huggy2" --repo-id="[ваш ник на HF]/ppo-Huggy" --commit-message="Huggy"

## Смотрим итоги обучения собачки 🐕


- Открываем окружение: https://huggingface.co/spaces/ThomasSimonini/Huggy

- Нажимаем сыграть в нашу модель

- Выбираем все параметры настроек согласно нашему аккаунту в HF

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/load-huggy.jpg" alt="load-huggy" width="100%">